#0. lib 임포트, 데이터 로드

In [5]:
import pandas as pd
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings(action='ignore')

In [54]:
src_path = 'https://codepresso-online-platform-public.s3.ap-northeast-2.amazonaws.com/learning-resourse/python_da/kaggle_boston_price.csv'
df=pd.read_csv(src_path, sep=',')

In [9]:
df.head(3)

,TOWN,LON,LAT,CMEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,Nahant,-70.955,42.2550,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,Swampscott,-70.950,42.2875,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,Swampscott,-70.936,42.2830,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03


#1. 단순선형회귀분석

* 문제
  * 보스턴 집값 데이터셋을 활용하여 방의 갯수와 집값의 인과관계 회귀분석을 통해 검증

* 가설
  * H0 : 방의 갯수 ~ 집값 연관 o
  * H1 : 방의 갯수 ~ 집값 연관 x


In [13]:
# formula "Y ~ X" : 종속변수 ~ 독립변수
model = smf.ols(formula='CMEDV~RM', data =df).fit()

In [14]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  CMEDV   R-squared:                       0.485
Model:                            OLS   Adj. R-squared:                  0.484
Method:                 Least Squares   F-statistic:                     474.3
Date:                Wed, 05 Mar 2025   Prob (F-statistic):           1.31e-74
Time:                        06:41:17   Log-Likelihood:                -1671.6
No. Observations:                 506   AIC:                             3347.
Df Residuals:                     504   BIC:                             3356.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -34.6592      2.642    -13.118      0.0

#2. 다중선형회귀분석

* 문제
  * 지역정보, 위치정보 제외 칼럼들이 집값에 영향을 줄것인지 검증

* 가설
  * H0 : [TWON:LSTAT]~ 집값 연관 o
  * H1 : [TWON:LSTAT] ~ 집값 연관 x


In [33]:
import statsmodels.api as sm
import numpy as np

In [55]:
df.drop(['LON','LAT','TOWN'],inplace=True, axis='columns')

In [56]:
df.columns

Index(['CMEDV', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS',
       'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [58]:
cols = df.columns.tolist()
cols.remove('CMEDV')
df[cols]=(df[cols]-df[cols].mean(axis=0))/df[cols].std(axis=0)

In [59]:
cols

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

In [60]:
df.head(3)

,CMEDV,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,24.0,-0.419367,0.284548,-1.286636,-0.272329,-0.144075,0.413263,-0.119895,0.140075,-0.981871,-0.665949,-1.457558,0.440616,-1.074499
1,21.6,-0.416927,-0.487240,-0.592794,-0.272329,-0.739530,0.194082,0.366803,0.556609,-0.867024,-0.986353,-0.302794,0.440616,-0.491953
2,34.7,-0.416929,-0.487240,-0.592794,-0.272329,-0.739530,1.281446,-0.265549,0.556609,-0.867024,-0.986353,-0.302794,0.396035,-1.207532


In [61]:
Y = df['CMEDV']
X = df[cols]

In [62]:
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  CMEDV   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     110.2
Date:                Wed, 05 Mar 2025   Prob (F-statistic):          1.83e-136
Time:                        06:57:46   Log-Likelihood:                -1494.2
No. Observations:                 506   AIC:                             3016.
Df Residuals:                     492   BIC:                             3076.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.5289      0.209    107.762      0.0

age와 indus는 유의사실보다 높게 나왔다 -> 이를 빼고 다시 다중선형회귀분석을 진행해야한다.

In [64]:
cols.remove('AGE')
cols.remove('INDUS')

In [65]:
Y = df['CMEDV']
X = df[cols]

In [66]:
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  CMEDV   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.739
Method:                 Least Squares   F-statistic:                     130.8
Date:                Wed, 05 Mar 2025   Prob (F-statistic):          1.50e-138
Time:                        06:59:53   Log-Likelihood:                -1494.3
No. Observations:                 506   AIC:                             3013.
Df Residuals:                     494   BIC:                             3063.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.5289      0.209    107.964      0.0

별..차이는 없는듯..
